In [1]:
##script to run caiman cnmfe 
#pass folder containing motion corrected mmpap file concactenated in C order as sys arg 1
#motion corrected mmpap file concactenated in C order as sys arg 2

##!/Users/johnmarshall/miniconda3/envs/caiman/bin/python3
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import logging
import matplotlib.pyplot as plt
import numpy as np

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2
import sys
import h5py

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
bpl.output_notebook()

#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

Loading BokehJS ...

In [46]:
from tqdm import tqdm

In [2]:
## path to motion corrected tif tif file
folder = '/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/Analysisrunonsubstack_1/'
mc_file = 'memmap__d1_480_d2_752_d3_1_order_C_frames_101_.mmap'
#folder = sys.arvg[1]
#mc_file = sys.arvg[2]
memory_map_file = folder+mc_file

In [3]:
print(memory_map_file)

/Volumes/My_Passport/MiniscopeMovies/caiman_pipeline_test/Analysisrunonsubstack_1/memmap__d1_480_d2_752_d3_1_order_C_frames_101_.mmap


In [4]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(memory_map_file)
images = Yr.T.reshape((T,) + dims, order='F')

In [5]:
plt.imshow(images[100])
plt.show()

     5879437 [_base.py:_update_title_position():2515] [3557] update_title_pos
     5879496 [font_manager.py:    _findfont_cached():1254] [3557] findfont: Matching :family=sans-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf') with score of 0.050000.


In [6]:
#
opts = params.CNMFParams(params_dict={})
#
bord_px = 0
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None
gSig = (3, 3)       # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = (13, 13)     # average diameter of a neuron, in general 4*gSig+1
Ain = None          # possibility to seed with predetermined binary masks
merge_thresh = .7   # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 0             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .2       # min peak value from correlation image
min_pnr = 4        # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor

opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thresh': merge_thresh,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px})                # number of pixels to not consider in the borders)

     5900506 [params.py:                 set():780] [3557] Changing key rf in group patch from None to 40
     5900507 [params.py:                 set():780] [3557] Changing key stride in group patch from None to 20
     5900508 [params.py:                 set():780] [3557] Changing key nb_patch in group patch from 1 to 0
     5900509 [params.py:                 set():780] [3557] Changing key low_rank_background in group patch from True to None
     5900509 [params.py:                 set():780] [3557] Changing key del_duplicates in group patch from False to True
     5900510 [params.py:                 set():780] [3557] Changing key p in group preprocess from 2 to 1
     5900511 [params.py:                 set():780] [3557] Changing key method_init in group init from greedy_roi to corr_pnr
     5900511 [params.py:                 set():780] [3557] Changing key K in group init from 30 to None
     5900513 [params.py:                 set():780] [3557] Changing key gSig in group init fro

In [7]:
# compute some summary images (correlation and peak to noise)
cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile

In [13]:
inspect_correlation_pnr(cn_filter, pnr)

     6141286 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1396a3240>
     6141287 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6141288 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1396a3240>
     6141289 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6141320 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x131385780>
     6141321 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6141322 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x131385780>
     6141323 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6142194 [_base.py:_update_title_position():2515] [3557] update_title_pos
  

     6146801 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6146802 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x13276e6d8>
     6146802 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6146808 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6146834 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6146847 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6146870 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6146882 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6146885 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6146889 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6146892 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6147726 [colorbar.py:_get_ticker_locator_formatter():50

     6148741 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6148752 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6148756 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6148760 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6148763 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6148786 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284f2b38>
     6148787 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6148787 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284f2b38>
     6148789 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6148795 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284164e0>
     6

     6149988 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6149990 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x12811fe48>
     6149990 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6149998 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x12811f7f0>
     6149999 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6149999 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x12811f7f0>
     6150000 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6150005 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6150031 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6150044 [_base.py:_update_title_position():2515] [3557] update_

     6151259 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1326ce080>
     6151259 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6151264 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6151289 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6151302 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6151328 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6151338 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6151342 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6151346 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6151350 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6151373 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x132472240>
     6151374 [col

...

     6155352 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6155377 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6155393 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6155418 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6155429 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6155432 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6155436 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6155440 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6155464 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x132456908>
     6155464 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6155465 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1324569

     6157166 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6157173 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284ba9e8>
     6157174 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6157174 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284ba9e8>
     6157175 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6157180 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284f2eb8>
     6157180 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6157181 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284f2eb8>
     6157182 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
  

     6157805 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284ba080>
     6157806 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6157807 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x1284ba080>
     6157808 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6157822 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6157846 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6157862 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6157889 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6157899 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6157903 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6157907 [_base.py:_update_title_position():2515] [3557] update_titl

     6160127 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6160142 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6160165 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6160179 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6160183 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6160186 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6160190 [_base.py:_update_title_position():2515] [3557] update_title_pos
     6160214 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x128139048>
     6160215 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6160215 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x128139048>
     6160216 [colorbar.py:         _add_solids():693] [3557] Setting pcolo

     6160838 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x128115c18>
     6160839 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6160839 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x128115c18>
     6160840 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6160845 [colorbar.py:_get_ticker_locator_formatter():507] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x12848aa58>
     6160846 [colorbar.py:        update_ticks():536] [3557] Using auto colorbar locator on colorbar
     6160847 [colorbar.py:        update_ticks():537] [3557] locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x12848aa58>
     6160847 [colorbar.py:         _add_solids():693] [3557] Setting pcolormesh
     6160852 [_base.py:_update_title_position():2515] [3557] update_title_pos
  

In [11]:
# print parameters set above, modify them if necessary based on summary images
print(min_corr) # min correlation of peak (from correlation image)

0.2


In [12]:
print(min_pnr)  # min peak to noise ratio

4


In [14]:
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)

print('running cnmfe')
cnm.fit(images)

/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/numpy/core/memmap.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  res = super(memmap, self).__getitem__(index)
     6183609 [params.py:                 set():780] [3557] Changing key init_batch in group online from 200 to 101
     6183610 [cnmf.py:                 fit():404] [3557] (101, 480, 752)
     6183611 [params.py:                 set():780] [3557] Changing key medw in group spatial from None to (3, 3)
     6183628 [params.py:                 set():780] [3557] Changing key se in group spatial from None to [[1 1 1]
 [1 1 1]
 [1 1 1]]
     6183629 [params.py:                 set():780] [3557] Changing key ss in group spatial from None to [[1 1 1]
 [1 1 1]
 [1 1 1]]
     6183629 [cn

running cnmfe
(80, 80)
1685.3500788211823
Transforming patches into full matrix
Skipped %d Empty Patch 0
Generating background
Generating background DONE
merging


/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/caiman/source_extraction/cnmf/merging.py:141: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  C_corr[ii, overlap_indeces] = corr_values


[33 35]
[14 20]


/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/scipy/signal/spectral.py:1970: UserWarning: nperseg = 256 is greater than input length  = 101, using nperseg = 101
  .format(nperseg, input_length))
/Users/johnmarshall/miniconda2/envs/caiman/lib/python3.6/site-packages/caiman/source_extraction/cnmf/deconvolution.py:991: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  g = np.linalg.lstsq(A, xc[lags + 1:])[0]


No neurons merged!


In [15]:
# get contours of found components
Cn = cm.local_correlations(images.transpose(1,2,0))
Cn[np.isnan(Cn)] = 0

In [16]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 3            # adaptive way to set threshold on the transient size
r_values_min = 0.85    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': False})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

    81633160 [params.py:                 set():780] [3557] Changing key min_SNR in group quality from 2.5 to 3
    81633162 [params.py:                 set():780] [3557] Changing key rval_thr in group quality from 0.8 to 0.85
    81633164 [params.py:                 set():780] [3557] Changing key use_cnn in group quality from True to False
    81633174 [components_evaluation.py:estimate_components_quality():682] [3557] EVALUATING IN PARALLEL... NOT RETURNING ERFCs


 ***** 
Number of total components:  60
Number of accepted components:  24


In [17]:
cnm.estimates.plot_contours(img=cn_filter, idx=cnm.estimates.idx_components)

TypeError: object of type 'NoneType' has no len()

In [22]:
hf = h5py.File(folder + 'output_data.h5', 'w')
hf.create_dataset('A', data=cnm.estimates.A.toarray())
hf.create_dataset('C', data=cnm.estimates.C)
hf.close()

In [34]:
cnm.__dict__['dims'] = np.array(cnm.__dict__['dims'])

In [37]:
cnm.save(folder + 'cnm_output.hdf5')

    33697872 [utils.py:recursively_save_dict_contents_to_group():440] [2039] dxy is a tuple ****
    33697922 [utils.py:recursively_save_dict_contents_to_group():440] [2039] gSig is a tuple ****


ValueError: Cannot save <class 'tuple'> type.

In [38]:
cnm.__dict__

{'dview': <multiprocessing.pool.Pool at 0x129fd6898>,
 'dims': array([480, 752]),
 'skip_refinement': False,
 'remove_very_bad_comps': False,
 'params': <caiman.source_extraction.cnmf.params.CNMFParams at 0x10fd4e400>,
 'estimates': <caiman.source_extraction.cnmf.estimates.Estimates at 0x11f6624e0>}

In [26]:
plt.plot(cnm.estimates.C[1])
plt.show()

    33327075 [_base.py:_update_title_position():2515] [2039] update_title_pos


In [23]:
cnm.estimates

In [43]:
np.shape(cnm.estimates.C)

(1, 200)

In [71]:
#plot all deconvolved spike trains 
for cell in range(np.shape(cnm.estimates.S)[0]):
    plt.plot(cnm.estimates.S[cell])
plt.show()

    85482174 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485095 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485154 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485221 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485296 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485375 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485570 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485637 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485706 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85485939 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85486015 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85486113 [_base.py:_update_title_position():2515] [3557] update_title_pos
    85486218 [_base.py:_update_title_position():2515] [3557] upd

In [56]:
cnm.estimates.A

<360960x60 sparse matrix of type '<class 'numpy.float64'>'
	with 18951 stored elements in Compressed Sparse Column format>

In [61]:
A = cnm.estimates.A.toarray()

In [62]:
# A is normalized to 1 for display
#A -= np.vstack(A.min(1))
#A /= np.vstack(A.max(1))
Atotal = A.sum(1).reshape(752, 480)

In [67]:
plt.imshow(np.transpose(Atotal))
plt.show()

    83601864 [_base.py:_update_title_position():2515] [3557] update_title_pos
    83662420 [_base.py:_update_title_position():2515] [3557] update_title_pos
    83663344 [_base.py:_update_title_position():2515] [3557] update_title_pos


In [44]:
np.shape(A)

(360960, 60)

In [35]:
np.size(A[:, 0])

360960

In [52]:
cell_1 = np.reshape(A[:,1], (752, 480))

In [51]:
for cell in range(np.shape(A)[1]):
    plt.imshow(np.reshape(A[:,cell], (752, 480)))
plt.show()

    82899561 [_base.py:_update_title_position():2515] [3557] update_title_pos


In [53]:
plt.imshow(cell_1.transpose())
plt.show()

    82992641 [_base.py:_update_title_position():2515] [3557] update_title_pos


In [83]:
##Display contours and denoised traces 
plt.subplot(2,1,1)
plt.imshow(np.reshape(A[:,cell], (752, 480)).transpose())
plt.subplot(2,1,2)
plt.plot(cnm.estimates.S[cell])
plt.draw()
plt.pause(1)
input("<Hit Enter To Close>")
plt.close()

    86691901 [_base.py:_update_title_position():2515] [3557] update_title_pos
    86691938 [_base.py:_update_title_position():2515] [3557] update_title_pos


<Hit Enter To Close>
